# Linear Regression

In [ ]:
from plotly.graph_objects import Figure, Scatter, Image, Heatmap, Surface
from plotly.express       import imshow
from plotly.subplots      import make_subplots
from math                 import sqrt, pi, log, exp
from numpy                import array, linspace, logspace, arange, eye, zeros, trace, zeros_like
import numpy              as np
from numpy.random         import random, randn, seed
from numpy.linalg         import inv, solve, det
from scipy.stats          import multivariate_normal
from scipy.optimize       import minimize
rand = random

## Model specification

In [ ]:
# define the features for polynomials
# `x` must be a vector
def phi(x, d):
    return x.reshape(x.shape[0],1) ** arange(d).reshape(1,d)
phi(randn(5), 3)       # test

In [ ]:
def f(x, w):
    d = w.shape[0]
    return phi(x, d) @ w
f(randn(5), randn(3))

In [ ]:
def sample(w_true, n, noise_std=0.05):
    x = rand(n)                   # uniformly distributed locations
    y = f(x, w_true) + noise_std*randn(n)  # noisy measurements
    return x, y
sample(array([0.1, 0.2]), 5)

In [ ]:
def linear_regression(x, y, d):
    X = phi(x, d)
    # maximum likelihood estimator
    #w_ml = inv(X.T @ X) @ X.T @ y
    # x = inv(A)@y
    # x = solve(A, y)
    w_ml = solve(X.T @ X, X.T @ y) # same as previous line
    return w_ml

In [ ]:
# implement mean-squared error
def mse(x, y, w):
    return ((y-f(x,w))**2).mean()

## Toy example

In [ ]:
# create some toy data
seed(0)          # set the seed for numpy
n_train = 20
#n_test  = 50
sigma   = 0.2
w_true  = array([0.1, -2.5, 3.0])
print(f"w_true = {w_true}")
x_train, y_train = sample(w_true, n_train, sigma)
x_pred  = linspace(0.0, 1.0, n_test)
# show the results for different `d`
d_candidates = [2, 3, 4, 10, 18]
fig = Figure()
fig.add_trace(Scatter(x=x_train, y=y_train, mode='markers', name='train'))
fig.add_trace(Scatter(x=x_pred, y=f(x_pred, w_true), mode='lines', name='true', line=dict(dash='dot')))
for d in d_candidates:
    w_ml    = linear_regression(x_train, y_train, d)
    y_pred  = f(x_pred, w_ml)
    fig.add_trace(Scatter(x=x_pred,  y=y_pred,  mode='lines', name=f'd={d}'))
    fig.update_layout(title='Linear regression', xaxis_title='x', yaxis_title='y')
fig.show()

In [ ]:
# 1st attempt: evaluating on the training to choose d
for d in range(1,20):
    w = linear_regression(x_train, y_train, d)
    print(f"d={d} MSE={mse(x_train, y_train, w)}")


In [ ]:
# 2nd attempt: evaluate on a evaluation set to choose d
n_eval = 100
x_eval, y_eval = sample(w_true, n_eval, sigma)
for d in range(1,10):
    w = linear_regression(x_train, y_train, d)
    print(f"d={d} MSE={mse(x_eval, y_eval, w)}")
d_best = 3   # chosen by looking at the numbers

In [ ]:
# 3rd attempt: calculate the test error for the best d
n_test = 100
x_test, y_test = sample(w_true, n_test, sigma)
w = linear_regression(x_train, y_train, d_best)
print(f"d={d_best} MSE={mse(x_test, y_test, w)}")

Observe that it is slightly larger than the evaluation error for $d=3$.

# Bayesian linear regression

In [ ]:
# create some toy data
seed(0)          # set the seed for numpy
d = 2
n_train = 100
n_test  = 50
sigma2  = 0.04
Sigma   = sigma2 * eye(n_train)   # measurement noise
w_true  = 2.0 * randn(d)            # true weights
x_train, y_train = sample(w_true, n_train, sqrt(sigma2))
x_pred  = linspace(0.0, 1.0, n_test)

In [ ]:
y_train.shape

In [ ]:
# prior for `w`
d  = 2
w0 = zeros(d)
tau2 = 1.0    # variance
V0 = tau2 * eye(d)

In [ ]:
# posterior for `w` after seeing data points `x`, `y`
# `x` and `y` must have shape (n,)
def posterior_w(x, y, w0, V0, Sigma):
    d = w0.shape[0]
    X = phi(x, d)
    invV0    = inv(V0)
    invSigma = inv(Sigma)
    Vn = inv(X.T @ invSigma @ X + invV0)
    wn = Vn @ (invV0 @ w0 + X.T @ invSigma @ y)
    return wn, Vn
posterior_w(x_train, y_train, w0, V0, Sigma)

## Implement log marginal likelihood

In [ ]:
def log_marginal_likelihood(y, x, d, theta):
    sigma2, tau2 = theta[0], theta[1]     # unpack
    n = y.shape[0]
    X = phi(x, d)
    A = sigma2*eye(n) + tau2* X @ X.T
    #return -0.5* y @ inv(A) @ y - 0.5*log(det(A)) - 0.5*n*log(2*pi)
    return -0.5* y @ solve(A, y) - 0.5*log(det(A)) - 0.5*n*log(2*pi)
lml = lambda theta: -log_marginal_likelihood(y_train, x_train, d, theta)

In [ ]:
# calculate the log marginal likelihood on a grid
d = 2
m = 20
sigma2s = logspace(-3, 2, m)
tau2s   = logspace(-3, 2, m)
lml_values = array([[lml(array([sigma2, tau2])) for sigma2 in sigma2s] for tau2 in tau2s])
#imshow(np.log(lml_values))
fig = Figure(data=[Surface(z=lml_values)])
fig.show()

In [ ]:
sigma2s[6], tau2s[15]

## Implement derivative of log marginal likelihood

In [ ]:
def grad_log_marginal_likelihood(y, x, d, theta):
    sigma2, tau2 = theta[0], theta[1]     # unpack
    n = y.shape[0]
    y = y.reshape(n, 1)
    X = phi(x, d)
    A = sigma2*eye(n) + tau2* X @ X.T
    B = inv(A)
    BYYB = 0.5 * (B @ y @ y.T @ B - B)
    grad0 = trace(BYYB)
    grad1 = trace(BYYB @ X @ X.T)
    return array([grad0, grad1])

In [ ]:
def finite_diff(f, x, delta):
    """estimate the gradient by finite-differencing method"""
    grad_f, dx = zeros_like(x), zeros_like(x)
    for i in range(x.size):
        dx.flat[i]     = delta
        grad_f.flat[i] = f(x+dx) - f(x-dx)
        dx.flat[i]     = 0.0
    return grad_f / (2*delta)

In [ ]:
grad_lml = lambda theta: -grad_log_marginal_likelihood(y_train, x_train, d, theta)
theta = rand(2)
print(finite_diff(lml, theta, 1e-5))
print(grad_lml(theta))

## Implement gradient descent

In [ ]:
def gradient_descent(f, grad_f, x0, learning_rate=0.1, maxiter=100):
    x  = x0
    for iter in range(maxiter):
        x = x - learning_rate * grad_f(x)
        print(f(x), x)
    return x

In [ ]:
# test gradient_descent
g = lambda x: x**2
grad_g = lambda x: 2*x
gradient_descent(g, grad_g, 20.0)

## Apply gradient descent to marginal likelihood

In [ ]:
d = 3
log_marginal_likelihood(y_train, x_train, d, array([0.04, 3.0]))

In [ ]:
theta0 = array([0.1, 0.1])
gradient_descent(lml, grad_lml, theta0, learning_rate=0.0001, maxiter = 100)

In [ ]:
log_marginal_likelihood(y_train, x_train, d, theta0)